In [45]:
from pathlib import Path
import pickle

import pandas
from pandas import DataFrame
import yaml


def log_as_dataframe(log):
    columns = ['timestamp', 'level', 'source', 'function', 'data']
    df = pandas.DataFrame(log, columns=columns)
    return df


df = log_as_dataframe(pickle.load(open('../log.pkl', 'rb')))
df = df[df['level'] == 'DATA']
df.head()

,timestamp,level,source,function,data
0,55311.0,DATA,,,"[0.000,0.000,0.000,0.490,0.465,0.111,0.099,0.0..."
1,55313.0,DATA,,,"[0.000,0.000,0.000,0.490,0.489,0.111,0.100,0.0..."
2,55315.0,DATA,,,"[0.000,0.000,0.000,0.490,0.489,0.111,0.099,0.0..."
3,55317.0,DATA,,,"[0.000,0.000,0.000,0.490,0.465,0.112,0.100,0.0..."
4,55319.0,DATA,,,"[0.000,0.000,0.000,0.490,0.489,0.111,0.099,0.0..."


In [46]:
data = []
for row in df.itertuples():
    try:
        value = yaml.load(row.data)
    except yaml.error.YAMLError:
        continue
    if len(value) != 15:
        print(len(value), value)
        continue
    data.append([row.timestamp] + value)
df = DataFrame(data)
df.columns = ['timestamp', 'side_integral', 'front_integral', 'diagonal_integral', 'front_left', 'front_right', 'side_left', 'side_right',
              'linear_ideal', 'linear_measured', 'angular_ideal', 'angular_measured',
              'voltage_left', 'voltage_right', 'pwm_left', 'pwm_right']
df['timestamp'] /= 1000
df = df.set_index('timestamp')
df = df.sort_index()
df.head()

/container/miniconda/envs/bulebule/lib/python3.6/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


20 [0.0, 0.0, 0.0, 0.49, 0.489, '0.0.000', 0.0, 0.0, 0.49, 0.489, 0.111, 0.099, 0.0, 0.0, 0.0, -0.005, 0.0, 0.0, 0, 0]
20 [0.0, 0.0, 0.0, 0.49, 0.465, 0.111, 0.1, 0.0, 0.0049, 0.489, 0.111, 0.1, 0.0, 0.0, 0.0, -0.005, 0.0, 0.0, 0, 0]
10 [0.0, 0.0, 0.0, 0.49, 0.489, 5, 0.0, 0.0, 0, 0]
62 0.223,0.204,0.119,0.659,0.653,-4.281,-4.399,2.11,2.40,239,273]
17 [0.0, 0.0, 0.0, 0.404, 0.445, 0.082, 0.088, -0.136, -0.129, 0.0, -0.012, -1.0, -0.045, -1.22, -1.11, -139, -126]


,side_integral,front_integral,diagonal_integral,front_left,front_right,side_left,side_right,linear_ideal,linear_measured,angular_ideal,angular_measured,voltage_left,voltage_right,pwm_left,pwm_right
timestamp,,,,,,,,,,,,,,,
55.311,0.0,0.0,0.0,0.49,0.465,0.111,0.099,0.0,0,0.0,-0.005,0.0,0.0,0,0
55.313,0.0,0.0,0.0,0.49,0.489,0.111,0.100,0.0,0,0.0,-0.007,0.0,0.0,0,0
55.315,0.0,0.0,0.0,0.49,0.489,0.111,0.099,0.0,0,0.0,-0.001,0.0,0.0,0,0
55.317,0.0,0.0,0.0,0.49,0.465,0.112,0.100,0.0,0,0.0,-0.009,0.0,0.0,0,0
55.319,0.0,0.0,0.0,0.49,0.489,0.111,0.099,0.0,0,0.0,-0.005,0.0,0.0,0,0


In [47]:
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure


output_notebook()

Loading BokehJS ...

In [48]:
def show_data(df):
    integral = figure(width=800, height=400,
                     title='Integral errors log',
                     x_axis_label='Time (s)',
                     y_axis_label='Error')
    integral.line(x=df.index, y=df['side_integral'], color='orange', legend='Side integral')
    integral.line(x=df.index, y=df['front_integral'], color='green', legend='Front integral')
    integral.line(x=df.index, y=df['diagonal_integral'], legend='Diagonal integral')
    
    sensors = figure(width=800, height=400,
                     title='Front sensors log',
                     x_axis_label='Time (s)',
                     y_axis_label='Distance (m)',
                     y_range=(0., 0.4))
    sensors.line(x=df.index, y=df['front_left'], color='orange', legend='Front-left')
    sensors.line(x=df.index, y=df['front_right'], legend='Front-right')

    linear = figure(width=800, height=400,
                    title='Linear speed',
                    x_axis_label='Time (s)',
                    y_axis_label='Speed (m/s)',
                    x_range=sensors.x_range)
    linear.line(x=df.index, y=df['linear_ideal'], color='orange', legend='Ideal')
    linear.line(x=df.index, y=df['linear_measured'], legend='Measured')

    angular = figure(width=800, height=400,
                     title='Angular speed',
                     x_axis_label='Time (s)',
                     y_axis_label='Speed (rad/s)',
                     x_range=sensors.x_range)
    angular.line(x=df.index, y=df['angular_ideal'], color='orange', legend='Ideal')
    angular.line(x=df.index, y=df['angular_measured'], legend='Measured')

    pwm = figure(width=800, height=400,
                 title='Output PWM',
                 x_axis_label='Time (s)',
                 y_axis_label='Duty',
                 x_range=sensors.x_range)
    pwm.line(x=df.index, y=df['pwm_left'], color='orange', legend='Left')
    pwm.line(x=df.index, y=df['pwm_right'], legend='Right')

    grid = gridplot([integral, sensors, linear, angular, pwm], ncols=1)

    show(grid)


show_data(df)